In [ ]:
%load_ext autoreload
%autoreload 2

from tcd_pipeline.modelrunner import ModelRunner
import ttach as tta
from PIL import Image
import numpy as np
import os

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
runner = ModelRunner("../config/base_semantic_segmentation.yaml")
runner.model.model = tta.SegmentationTTAWrapper(runner.model.model, transforms=tta.aliases.d4_transform())

In [ ]:
image_path = "../data/5c15321f63d9810007f8b06f_10_00000.tif"
results = runner.predict(image_path, warm_start = True)

In [ ]:
print(results)

In [ ]:
results.save_masks(output_path="./results", image_path=image_path)

In [ ]:
results.visualise(output_path="./results")

In [ ]:
!pip install python-liquid

In [ ]:
report_results = {
    "raw_image": "raw_image.jpg",
    "canopy_mask": "canopy_mask.jpg",
    "canopy_confidence": "confidence_map.jpg",
    "local_maximum": "local_maximum.jpg",
    "canopy_cover": results.canopy_cover,
    "image_name": os.path.basename(results.image.name),
    "image_res": results.image.res[0],
    "image_crs": results.image.crs,
}

In [ ]:
from liquid import Environment
from liquid import Mode
from liquid import StrictUndefined
from liquid import FileSystemLoader

env = Environment(
    tolerance=Mode.STRICT,
    undefined=StrictUndefined,
    loader=FileSystemLoader("../templates/"),
)

template = env.get_template("canopy_coverage.html")
result = template.render(data=report_results, page_title="Canopy coverage report")

with open("./results/report.html", 'w') as fp:
    fp.write(result)